In [1]:
:opt no-lint
{-# LANGUAGE TypeSynonymInstances FlexibleInstances #-}

이제 그림 \ref{fig:bigStepCBV}와 \ref{fig:bigStepCBN}의 큰걸음 의미구조를 하스켈로 옮겨 보자.

In [2]:
type Nm = String -- 변수 이름은 문자열로
data Expr = Var Nm         -- x
          | Lam Nm Expr    -- (λx.e)
          | App Expr Expr  -- (e1 e2)
          deriving (Eq, Ord)

In [3]:
instance Show Expr where -- 좀더 간결한 출력을 위해 직접 Show 인스턴스 선언
  showsPrec _ (Var x) = showString x
  showsPrec p (Lam x e) = showParen (p > 1) $
    showString ("\\lambda "++x++".") . showsPrec 1 e
  showsPrec p (App e1 e2) = showParen (p > 9) $
    showsPrec 9 e1 . showString "\\;" . showsPrec 10 e2

import IHaskell.Display (latex)
instance IHaskellDisplay Expr where
    display e = display [latex $ "$"++show e++"$"]

In [4]:
Lam "x" (Lam "y" (App (Var "x") (Lam "z"  (Var "y"))))

In [5]:
-- 연산자 (.), ($) 및 역따옴표 중위 표현 활용 
Lam "x" . Lam "y" $ Var "x" `App` Lam "z" (Var "y")

In [6]:
type Env = [(Nm,Clos)] -- 값계산 환경
data Clos = Cl Expr Env deriving (Eq,Ord,Show) -- 클로저

In [7]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Env where
    show env = "\\{"++ intercalate ",\\," (map show env) ++"\\}"
instance {-# OVERLAPS #-} Show (Nm,Clos) where
    show (x,cl) = "\\langle "++x++"{\\mapsto}"++show cl++"\\rangle "
instance Show Clos where
    show (Cl e env) = "\\langle "++show e++","++show env++"\\rangle "

instance {-# OVERLAPS #-} IHaskellDisplay Env where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Clos) where
    display e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Clos where
    display e = display [latex $ "$"++show e++"$"]

In [8]:
sigma01 = [("f", Cl (Lam "z" $ Var "z") [])]

[ ( "x", Cl (Lam "z" $ Var "z")               []     ),
  ( "y", Cl (Lam "x" $ Var "f" `App` Var "x") sigma01) ]

In [9]:
mapM display it

# 값계산 환경을 활용한 작은걸음 의미구조
TODO TODO

%% https://matt.might.net/articles/cek-machines/

TODO TODO

\begin{align}
\langle\sigma,x\rangle;\kappa
        &\longmapsto
\sigma(x);\kappa
\\
\langle\sigma,e_1~e_2\rangle;\kappa
        &\longmapsto
\langle\sigma,e_1\rangle;\langle\sigma,\bullet~e_2\rangle,\kappa
\\
\langle\sigma_1,\lambda x.e\rangle;\langle\sigma,\bullet~e_2\rangle,\kappa
        &\longmapsto
\langle\sigma,e_2\rangle;\langle\sigma_1,(\lambda x.e)\,\bullet\rangle,\kappa
\\
v_2;\langle\sigma_1,(\lambda x.e)\,\bullet\rangle,\kappa
        &\longmapsto
\langle\{x{\mapsto}v_2\}\sigma_1,e\rangle;\kappa
\end{align}

\begin{align}
\langle\sigma,x\rangle;\kappa
        &\longmapsto
\sigma(x);\kappa
\\
\langle\sigma,e_1~e_2\rangle;\kappa
        &\longmapsto
\langle\sigma,e_1\rangle;\langle\sigma,\bullet~e_2\rangle,\kappa
\\
\langle\sigma_1,\lambda x.e\rangle;\langle\sigma,\bullet~e_2\rangle,\kappa
        &\longmapsto
\langle\{x{\mapsto}\langle\sigma,e_2\rangle\}\sigma_1,e\rangle;\kappa
\end{align}

\lipsum[1-3]